In [1]:
import Random

In [2]:
include("Storage.jl")
include("GillespieFunctions.jl")
include("Rates.jl")

tt_protein_fb_rates! (generic function with 1 method)

In [12]:
function balance_test(initialstate::Vector{Int64},params::Vector{Float64},numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG)
    statevector = initialstate
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    calcstor = BalanceEqData()

    endstate,steps = direct_gillespie!(initialstate,params,outcomes,tt_protein_fb_rates!,calcstor,updatestorage!,rng,numsteps,maxsteps)

    println("Flux Balance on x")
    println(relative_error(calcstor.meanRpX,calcstor.meanRmX))

    println("Flux Balance on y")
    println(relative_error(calcstor.meanRpY,calcstor.meanRmY))

    println("Cov Balance on x,x")
    println(cov_balance_comp(calcstor.CXRmX/calcstor.weightsum,calcstor.CXRmX/calcstor.weightsum,
                            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/calcstor.weightsum,calcstor.CXRpX/calcstor.weightsum))
    println("Cov Balance on y,y")
    println(cov_balance_comp(calcstor.CYRmY/calcstor.weightsum,calcstor.CYRmY/calcstor.weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/calcstor.weightsum,calcstor.CYRpY/calcstor.weightsum))
    println("Cov Balance on y,x")
    println(cov_balance_comp(calcstor.CYRmX/calcstor.weightsum,calcstor.CXRmY/calcstor.weightsum,
                            0,calcstor.CYRpX/calcstor.weightsum,calcstor.CXRpY/calcstor.weightsum))
    println(steps)
    println(endstate)
    println(calcstor.meanx)
    println(calcstor.meany)


    return calcstor
end

balance_test (generic function with 1 method)

In [14]:
calcs = balance_test([10,50],[10.,1.,4.,1.,-1.,50.],10^7,10^10,Random.Xoshiro())

Flux Balance on x
0.0003785245592879545
Flux Balance on y
0.00022101976757423308
Cov Balance on x,x
0.023911299392200755
Cov Balance on y,y
0.02025916664898015
Cov Balance on y,x
0.021533861160396104
100034590
[8, 24]
6.62374664466483
26.440466636341835


BalanceEqData(6.62374664466483, 26.440466636341835, 1.5127794362795355e6, 8.808344845843764e6, 9.47753503049765e7, 1.3626114786334293e7, 6.6092691905836345, 6.611771908631462, 26.447087634525847, 26.452934255906744, 8.665520906935355e6, -1.127534778819131e6, 1.4006779973914923e7, -7.966060620822394e6, 1.3276763060674649e7, 3.466208362774142e7, 9.41330944944052e7, 5.602711989565969e7)